In [1]:
from __future__ import print_function,division
import sys
sys.path.append('/home/oem/vision')
sys.path.append('/home/oem/miniconda3/lib/python3.9/site-packages/skimage')


import numpy as np
import random
import os
from os.path import  join
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torch.optim.lr_scheduler import ExponentialLR as ExponentialLR

import torchvision
import matplotlib.pyplot as plt

import medpy
import PIL
import json

from TMSNet_Train_Test import train, test,testWithAdversarialNoise
from TMSNet import TMSNet
import pdb




from DataLoader import get_data


random.seed(0)
torch.manual_seed(0)

device=torch.device("cuda:0")  

class option:
    def __init__(self,PathToTrainingFolder,PathToSaveTrainedModels,Dataset,AttackType,AttackNumber):
        
        self.PathToTrainingFolder = PathToTrainingFolder
        self.Dataset=Dataset       
        if self.Dataset=='Atrium2018':
            self.InputSize = 128           
            self.weight_decay=0.001 
            self.batchSize=8
        elif self.Dataset=='Atrium2013':
            self.InputSize = 192
            self.weight_decay=0.01 
            self.batchSize=4
            
        self.lr =0.0005
        self.AttackNumber= AttackNumber #3
        self.AttackType=AttackType #'FGSM'
        self.cuda = True
        self.nEpochs = 20
        self.gammaForScheduler=0.95 
        self.FolderName='8'
        self.threads=0
        self.testBatchSize=1
        self.PathToSaveTrainedModels=PathToSaveTrainedModels



        
class TrainingParameters:
    def __init__(self,device,training_data_loader_All,training_data_loader_A,training_data_loader_C,training_data_loader_S,val_data_loader_All,val_data_loader_A,val_data_loader_C,val_data_loader_S,testing_data_loader_S):
        
        self.initial_seed=23789 
        #self.optimizer=optimizer
        self.training_data_loader_All=training_data_loader_All
        self.training_data_loader_A=training_data_loader_A
        self.training_data_loader_C=training_data_loader_C
        self.training_data_loader_S=training_data_loader_S
             
        self.val_data_loader_All=val_data_loader_All           
        self.val_data_loader_A=val_data_loader_A
        self.val_data_loader_C=val_data_loader_C
        self.val_data_loader_S=val_data_loader_S
        self.testing_data_loader_S=testing_data_loader_S
        self.device=device
        





/home/oem/miniconda3/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [2]:
def checkpoint(epoch):
    model_out_path = join(opt.PathToSaveTrainedModels,"epoch_{}.pth".format(epoch))
    torch.save(model.state_dict(), model_out_path)
    
    
    fig, [ax1,ax2] = plt.subplots( nrows=1, ncols=2 )  # create figure & 1 axis
    ax1.plot(trainingLoss)
    ax1.set(xlabel='epochs', ylabel='TrainingLoss')  
    ax2.plot(validationLoss)
    ax2.set(xlabel='epochs', ylabel='ValidationLoss')  

    plt.savefig(join(opt.PathToSaveTrainedModels,"ErrorPlot_{}_Epoch.png".format(epoch)))
    plt.show()
    plt.close(fig)     
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [3]:
if __name__ == '__main__':



    model=TMSNet(W=10).to(device)
    
    ## update paths accordingly for your computer
    PathToSaveTrainedModels=join("/TrainedModels",str('2013'))
    PathToTrainingFolder = "/Training"
    
    
    
    Dataset='Atrium2013'         
    AttackType='FGSM' # or IterativeFGSM or Rician
    AttackNumber=0 #1 or 3
    
    
    opt=option(PathToTrainingFolder,PathToSaveTrainedModels,Dataset,AttackType,AttackNumber)
    optimizer  =optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=opt.lr) 
    scheduler=ExponentialLR(optimizer,gamma=opt.gammaForScheduler) 
    
    
    MainPath="/Datasets/Atrium2013/"
    dataPath=os.path.join(MainPath,'All')
    train_set=get_data(opt,dataPath,DataloaderType='training')
    training_data_loader_All = DataLoader(dataset=train_set, num_workers=0, batch_size=opt.batchSize, shuffle=True,pin_memory=True,drop_last=False)

    val_set = get_data(opt,dataPath,DataloaderType='validation')
    val_data_loader_All = DataLoader(dataset=val_set, num_workers=0, batch_size=opt.batchSize, shuffle=True,pin_memory=False)

    
    
    dataPath=os.path.join(MainPath,'A')
    train_set = get_data(opt,dataPath,DataloaderType='training')
    training_data_loader_A = DataLoader(dataset=train_set, num_workers=opt.threads, batch_size=opt.batchSize, shuffle=True,pin_memory=False,drop_last=False)

    val_set = get_data(opt,dataPath,DataloaderType='validation')
    val_data_loader_A = DataLoader(dataset=val_set, num_workers=0, batch_size=opt.batchSize, shuffle=True,pin_memory=False)


    ##########

    dataPath=os.path.join(MainPath,'C')
    train_set_C = get_data(opt,dataPath,DataloaderType='training')
    training_data_loader_C = DataLoader(dataset=train_set_C, num_workers=0, batch_size=opt.batchSize, shuffle=True,pin_memory=False,drop_last=False)


    val_set = get_data(opt,dataPath,DataloaderType='validation')
    val_data_loader_C = DataLoader(dataset=val_set, num_workers=0, batch_size=opt.batchSize, shuffle=True,pin_memory=False)



    dataPath=os.path.join(MainPath,'S')
    train_set_S = get_data(opt,dataPath,DataloaderType='training')
    training_data_loader_S = DataLoader(dataset=train_set_S, num_workers=0, batch_size=opt.batchSize, shuffle=True,pin_memory=False,drop_last=False)

    val_set = get_data(opt,dataPath,DataloaderType='validation')
    val_data_loader_S = DataLoader(dataset=val_set, num_workers=0, batch_size=opt.batchSize, shuffle=True,pin_memory=False)

    test_set_S =get_data(opt,dataPath,DataloaderType='test') 
    testing_data_loader_S = DataLoader(dataset=test_set_S, num_workers=0, batch_size=opt.testBatchSize, shuffle=False,pin_memory=False)

   
    TranPara=TrainingParameters(device,training_data_loader_All,training_data_loader_A,training_data_loader_C, training_data_loader_S,val_data_loader_All, 
                                val_data_loader_A,val_data_loader_C,val_data_loader_S,testing_data_loader_S)

 

    

    it=0

    dice=[]
    hfDistance=[]
    ASSD=[]
    Jaccard=[]

    

    trainingLoss=[]
    validationLoss=[]

    epsilon=0

    for epoch in range(opt.nEpochs):


        # for test with adversarial noise, please uncomment the part below
        ''' 
        if opt.AttackType=='Rician':
            noiseRange=[10,15,20,25]
        else:
            noiseRange=[0.01,0.02,0.03,0.04]

        for epsilon in noiseRange:

            dice=[]
            hfDistance=[]
            surface_distance=[]
            JaccardSimilarity=[]
            CosineSimilarities=[]
            
            dice,hfDistance,ASSD,Jaccard,CosineSimilarities=testWithAdversarialNoise(TranPara,opt,model,device,dice,hfDistance,ASSD,Jaccard,CosineSimilarities,epsilon)
            data={"dice":dice, "hfDistance":hfDistance,"ASSD":ASSD,"Jaccard":Jaccard,'CosineSimilarities'}
            #testWithAdversarialNoise
            print ("        ")
            print (" epsilon  ",epsilon)

            print ("        ")
            print(np.mean(ASSD),np.mean(hfDistance),np.mean(dice),np.mean(Jaccard),np.mean(CosineSimilarities))
            print(np.std(ASSD),np.std(hfDistance),np.std(dice),np.std(Jaccard),np.std(CosineSimilarities))
            with open(join(opt.PathToSaveTrainedModels,'PerformanceMetrics_'+opt.AttackType+'_'+str(opt.AttackNumber)+'_'+str(epsilon)+'.json'), 'w') as fp:
                json.dump(data, fp)


        pdb.set_trace()
        '''

       
        
        print(count_parameters(model))
        
        model,optimizer, trainLoss, valLoss= train(TranPara,opt,model,epoch,optimizer)
        trainingLoss.append(trainLoss)
        validationLoss.append(valLoss)
        scheduler.step()
        
        


        it=it+1
        print("TrainingEpoch:"+str(it))
        if epoch%1 == 0:
            checkpoint(epoch)
        if epoch%1 == 0 and epoch>9:

            dice=[]
            hfDistance=[]
            surface_distance=[]
            JaccardSimilarity=[]
            CosineSimilarities=[]

            dice,hfDistance,ASSD,Jaccard,CosineSimilarities=test(TranPara,opt,model,device,dice,hfDistance,ASSD,Jaccard,CosineSimilarities)

            data={"dice":dice, "hfDistance":hfDistance,"ASSD":ASSD,"Jaccard":Jaccard}
            print(np.mean(ASSD),np.mean(hfDistance),np.mean(dice),np.mean(Jaccard))
            print(np.std(ASSD),np.std(hfDistance),np.std(dice),np.std(Jaccard))
            with open(join(opt.PathToSaveTrainedModels,'PerformanceMetrics.json'), 'w') as fp:
                json.dump(data, fp)

       

In [ ]:
plt.plot(CosineSimilarities,Jaccard,'*')
plt.show()

In [ ]:
CosineSimilarities